In [1]:
import datetime
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"  # or choose another backend if you wish

from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, DatetimeTickFormatter
from bokeh.models.tools import HoverTool
from bokeh.io.export import get_screenshot_as_png
from bokeh.io import output_notebook
import pandas as pd
import sklearn
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

import keras
from keras.models import Sequential, Model
from keras.layers import Embedding, Flatten, Dense, Dropout, Input

import utils
import model_scoring

Using plaidml.keras.backend backend.


In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
train, test = utils.load_data(hourly=True)

Now, we will normalize the data.

In [4]:
mean = train.mean(axis=0)
std = train.std(axis=0)

utils.preprocess(train, mean, std)
utils.preprocess(test, mean, std)

labels = train.pop('target')
train.head()

,temp,dwpt,heat_idx,rh,pressure,vis,wc,wdir,wspd,prcp,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
2014-09-25 06:00:00,63.0,57.0,63.0,81.0,30.36,10.0,63.0,50.0,20.0,0.0,...,0,0,0,0,0,0,1,0,0,0
2005-09-23 02:00:00,74.0,63.0,74.0,68.0,29.93,10.0,74.0,240.0,13.0,0.0,...,0,0,0,0,0,0,1,0,0,0
2006-07-31 02:00:00,79.0,66.0,81.0,64.0,29.83,10.0,79.0,60.0,5.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2008-07-12 01:00:00,76.0,57.0,76.0,52.0,30.08,10.0,76.0,190.0,8.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2011-03-05 09:00:00,45.0,40.0,45.0,82.0,30.40,10.0,39.0,180.0,12.0,0.0,...,1,0,0,0,0,0,0,0,0,0


We need validation data

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train, labels,
                                                      test_size=.2,
                                                      shuffle=True)

In [6]:
# build keras regressor
keras_reg = tf.keras.wrappers.scikit_learn.KerasRegressor(utils.build_model, input_shape=(X_train.shape[1],))

In [7]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

params = {
    "n_hidden": [1, 2, 3],
    "n_neurons": np.arange(16, 128),
    "learning_rate": reciprocal(3e-4, 3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, params, n_iter=10, cv=3)

In [ ]:
rnd_search_cv.fit(
    X_train, y_train, epochs=100,
    validation_data=(X_valid, y_valid), 
    callbacks=[keras.callbacks.EarlyStopping(patience=10)]
                 )

INFO:plaidml:Opening device "llvm_cpu.0"


Train on 63224 samples, validate on 23709 samples
Epoch 1/100
63224/63224 [==============================] - 9s 140us/step - loss: 1.0045 - val_loss: 1.5811
Epoch 2/100
63224/63224 [==============================] - 8s 130us/step - loss: 0.6226 - val_loss: 0.2585
Epoch 3/100
63224/63224 [==============================] - 8s 119us/step - loss: 0.5504 - val_loss: 0.2652
Epoch 4/100
63224/63224 [==============================] - 8s 119us/step - loss: 0.4829 - val_loss: 0.2001
Epoch 5/100
63224/63224 [==============================] - 8s 119us/step - loss: 0.4371 - val_loss: 0.4053
Epoch 6/100
63224/63224 [==============================] - 8s 129us/step - loss: 0.3781 - val_loss: 0.2219
Epoch 7/100
63224/63224 [==============================] - 8s 125us/step - loss: 0.3175 - val_loss: 0.1351
Epoch 8/100
63224/63224 [==============================] - 8s 119us/step - loss: 0.2829 - val_loss: 0.3091
Epoch 9/100
63224/63224 [==============================] - 8s 124us/step - loss: 0.2565 - val_

63224/63224 [==============================] - 11s 171us/step - loss: 0.0701 - val_loss: 0.0787
Epoch 34/100
63224/63224 [==============================] - 10s 153us/step - loss: 0.0683 - val_loss: 0.0740
Epoch 35/100
63224/63224 [==============================] - 10s 163us/step - loss: 0.0669 - val_loss: 0.0864
Epoch 36/100
63224/63224 [==============================] - 10s 154us/step - loss: 0.0646 - val_loss: 0.0532
Epoch 37/100
63224/63224 [==============================] - 11s 179us/step - loss: 0.0628 - val_loss: 0.0912
Epoch 38/100
63224/63224 [==============================] - 9s 142us/step - loss: 0.0613 - val_loss: 0.0459
Epoch 39/100
63224/63224 [==============================] - 10s 152us/step - loss: 0.0606 - val_loss: 0.0479
Epoch 40/100
63224/63224 [==============================] - 11s 169us/step - loss: 0.0589 - val_loss: 0.0503
Epoch 41/100
63224/63224 [==============================] - 11s 176us/step - loss: 0.0583 - val_loss: 0.0664
Epoch 42/100
63224/63224 [=======

63224/63224 [==============================] - 14s 225us/step - loss: 0.0418 - val_loss: 0.0446
Epoch 60/100
63224/63224 [==============================] - 11s 174us/step - loss: 0.0414 - val_loss: 0.0410
Epoch 61/100
63224/63224 [==============================] - 12s 196us/step - loss: 0.0415 - val_loss: 0.0388
Epoch 62/100
63224/63224 [==============================] - 10s 165us/step - loss: 0.0409 - val_loss: 0.0383
Epoch 63/100
63224/63224 [==============================] - 11s 180us/step - loss: 0.0410 - val_loss: 0.0466
Epoch 64/100
63224/63224 [==============================] - 14s 220us/step - loss: 0.0405 - val_loss: 0.0608
Epoch 65/100
63224/63224 [==============================] - 12s 196us/step - loss: 0.0409 - val_loss: 0.0398
Epoch 66/100
63224/63224 [==============================] - 11s 175us/step - loss: 0.0404 - val_loss: 0.0408
Epoch 67/100
63224/63224 [==============================] - 11s 173us/step - loss: 0.0402 - val_loss: 0.0407
Epoch 68/100
63224/63224 [======

In [1]:
rnd_search_cv.best_params

NameError: name 'rnd_search_cv' is not defined